### Note: 
Make sure you go through A0_Initialization first before running the following file.

In [15]:
import math
import time
import os
import cv2 
import keyboard
import pickle
import numpy as np
from PIL import Image, ImageOps, ImageGrab
from matplotlib import pyplot as plt
from skimage import io, color, measure, morphology
from scipy import ndimage
from scipy.ndimage import rotate
import copy
%matplotlib inline

In [23]:
# Load the player_circle_center from the file.
with open('player_circle_center_1.pkl', 'rb') as file:
    player_circle_center = pickle.load(file)
with open('sscoords.pkl', 'rb') as file:
    sscoords = pickle.load(file)
with open('chatcoords.pkl', 'rb') as file:
    chatcoords = pickle.load(file)

furthest_point_distance = np.sqrt((player_circle_center[0])**2 + (player_circle_center[1])**2) # Distance from player to corner of screen. This will be used to measure how far away enemies and items are.

In [25]:
def isColorOfItem(color):
    r, g, b = color
    r, g, b = int(r), int(g), int(b)
    diff = abs(r-g)
    return r > 150 and g > 150 and b < 15 and diff < 15

In [26]:
def GetItemsCount(original_img):
    player_circle_center_half = (player_circle_center[0]/2, player_circle_center[1]/2)
    posterized_image = ImageOps.posterize(original_img, 2)
    
    image_np = np.array(posterized_image)
    
    # Sizing Img Down
    scale_fraction = 0.2
    width = int(image_np.shape[1] * scale_fraction)
    height = int(image_np.shape[0] * scale_fraction)
    dim = (width, height)
    image_np = cv2.resize(image_np, dim)

    for i in range(image_np.shape[0]):  # Iterate over rows (height)
        for j in range(image_np.shape[1]):  # Iterate over columns (width)
            pixel = image_np[i, j]
            if (not isColorOfItem(pixel)) or np.sqrt((i - player_circle_center_half[0])**2 + (j - player_circle_center_half[1])**2) <= 10:
                pixel[0] = 0
                pixel[1] = 0
                pixel[2] = 0
            else:
                pixel[0] = 255
                pixel[1] = 255
                pixel[2] = 255
    
    # Removing small bits.
    deletion_range = (7, 7)
    size_threshold = 7 # The number of active pixels that have to be within the range around an active pixel.
    
    # Get the coordinates of the white pixels
    white_pixels_mask = (image_np[:, :, 0] == 255) & (image_np[:, :, 1] == 255) & (image_np[:, :, 2] == 255)
    white_pixel_coords = np.argwhere(white_pixels_mask)
    
    # Iterate over each white pixel's coordinates
    for x, y in white_pixel_coords:
        # Count the number of white pixels in the surrounding area
        active_neighbors = np.sum((abs(white_pixel_coords[:, 0] - x) < deletion_range[0]) & (abs(white_pixel_coords[:, 1] - y) < deletion_range[1]))
        # If the number of white neighbors is less than the minimum required, turn the pixel black
        if active_neighbors < size_threshold:
            image_np[x, y] = (0, 0, 0)  # Turn the pixel black
    
    image = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    
    # Check if the image is already in binary form, if not, apply a threshold
    # Assuming white pixel values are close to 255
    _, binary_image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    binary_image = cv2.dilate(binary_image, kernel, iterations=1)

    # Find connected components
    num_labels, labels = cv2.connectedComponents(binary_image)
    
    # The first label is the background, so the number of white pixel groupings is `num_labels - 1`
    return num_labels - 1

In [27]:
def GetEnemiesCount(original_img):
    image_np = np.array(original_img)
    
    # Sizing Img Down
    scale_fraction = 0.4
    width = int(image_np.shape[1] * scale_fraction)
    height = int(image_np.shape[0] * scale_fraction)
    dim = (width, height)
    image_np = cv2.resize(image_np, dim)
    
    for i in range(image_np.shape[0]):  # Iterate over rows (height)
        for j in range(image_np.shape[1]):  # Iterate over columns (width)
            pixel = image_np[i, j]
    
            if pixel[0] > 100 and pixel[0] > pixel[1]*2 and pixel[0] > pixel[2]*2:
                image_np[i, j] = np.array([255, 255, 255])
            else: 
                image_np[i, j] = np.zeros(3, dtype=np.uint8)  # Black
    
    image = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    
    # Check if the image is already in binary form, if not, apply a threshold
    # Assuming white pixel values are close to 255
    _, binary_image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)
    
    # Label the objects in the binary image
    labeled_mask, _ = ndimage.label(binary_image)
    
    # Find the properties of the labeled regions
    regions = measure.regionprops(labeled_mask)
    
    # Define a range for the circularity metric (0 for a line, 1 for a perfect circle)
    circularity_threshold = 0.3
    
    min_radius = 3
    
    regions_count = 0
    regions_centroids = []
    
    # Loop through region properties and filter out non-circular regions
    for region in regions:
        if region.area > 0 and region.perimeter > 0:
            # To measure the circularity of the region, we'll do the following:
            # Step 1, get all the points in the region.
            centroid = region.centroid
            coords = np.array(region.coords)
    
            # Step 2: Get the furthest point from the center.
            biggest_r = 0
            furthest_point = ()
            for coord in coords:
                distance = abs(centroid[0] - coord[0]) + abs(centroid[1] - coord[1])
                if distance > biggest_r:
                    biggest_r = distance
                    furthest_point = coord
    
            # Step 3: Compare the region area with a perfect circle area.
            circle_area = np.pi * (biggest_r ** 2)
    
            # Step 4: Calculate circularity
            circularity = region.area/circle_area
            
            # If the region is within the circularity thresholds and has at least the minimum diameter, keep it
            if circularity_threshold <= circularity and biggest_r >= min_radius:
                regions_count += 1
                regions_centroids.append(region.centroid)
    
    return regions_count

In [28]:
def toClockAngles(angle):
    clock_angles = [330, 300, 270, 240, 210, 180, 150, 120, 90, 60, 30, 0]
    closest_ind = 0
    for i in range(12):
        if np.abs(angle - clock_angles[i]) < np.abs(angle - clock_angles[closest_ind]):
            closest_ind = i

    if angle > 345:
        closest_ind = 11
    
    return closest_ind+1

In [29]:
def GetItems(original_img, player_direction_angle):
    player_circle_center_half = (player_circle_center[0]/2, player_circle_center[1]/2)
    
    posterized_image = ImageOps.posterize(original_img, 2)
    
    image_np = np.array(posterized_image)
    
    # Sizing Img Down
    scale_fraction = 0.2
    width = int(image_np.shape[1] * scale_fraction)
    height = int(image_np.shape[0] * scale_fraction)
    dim = (width, height)
    image_np = cv2.resize(image_np, dim)
    
    # Applying the mask.
    for i in range(image_np.shape[0]):  # Iterate over rows (height)
        for j in range(image_np.shape[1]):  # Iterate over columns (width)
            pixel = image_np[i, j]
            if not isColorOfItem(pixel) or np.sqrt((i - player_circle_center_half[0])**2 + (j - player_circle_center_half[1])**2) <= 10:
                pixel[0] = 0
                pixel[1] = 0
                pixel[2] = 0
            else:
                pixel[0] = 255
                pixel[1] = 255
                pixel[2] = 255
    
    # Removing small bits.
    deletion_range = (7, 7)
    size_threshold = 7 # The number of active pixels that have to be within the range around an active pixel.
    
    # Get the coordinates of the white pixels
    white_pixels_mask = (image_np[:, :, 0] == 255) & (image_np[:, :, 1] == 255) & (image_np[:, :, 2] == 255)
    white_pixel_coords = np.argwhere(white_pixels_mask)
    
    # Iterate over each white pixel's coordinates
    for x, y in white_pixel_coords:
        # Count the number of white pixels in the surrounding area
        active_neighbors = np.sum((abs(white_pixel_coords[:, 0] - x) < deletion_range[0]) & (abs(white_pixel_coords[:, 1] - y) < deletion_range[1]))
        # If the number of white neighbors is less than the minimum required, turn the pixel black
        if active_neighbors < size_threshold:
            image_np[x, y] = (0, 0, 0)  # Turn the pixel black
    
    image = cv2.cvtColor(image_np, cv2.COLOR_BGR2GRAY)
    
    # Check if the image is already in binary form, if not, apply a threshold
    # Assuming white pixel values are close to 255
    _, binary_image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)
    
    kernel = np.ones((5, 5), np.uint8)
    binary_image = cv2.dilate(binary_image, kernel, iterations=1)

    # Label the objects in the binary image
    labeled_mask, _ = ndimage.label(binary_image)
    
    # Find the properties of the labeled regions
    regions = measure.regionprops(labeled_mask)

    items = []
    
    for region in regions:
        centroid = region.centroid
        relative_centroid = (player_circle_center_half[0] - centroid[0], centroid[1] - player_circle_center_half[1])

        # Getting the item angle.
        angle_radians = math.atan2(relative_centroid[0], relative_centroid[1])
        direction_angle = math.degrees(angle_radians)
        if direction_angle < 0:
            direction_angle += 360

        # Getting the item's angle relative to the player
        relative_angle = direction_angle - player_direction_angle
        if relative_angle < 0:
            relative_angle += 360

        # Convert the angle to a clock angle (1-12)
        clock_angle = toClockAngles(relative_angle)

        # Getting the distance to enemy
        distance = np.sqrt((relative_centroid[0])**2 + (relative_centroid[1])**2)
        distance_perc = int((distance/furthest_point_distance) * 100)

        # Adding the item to the items array.
        items.append((clock_angle, distance_perc))

        # print("Item Center: {}".format(centroid))
        # print("Item Relative Center: {}".format(relative_centroid))
        # print("Item Angle: {}".format(direction_angle))
        # print("Relative Angle: {}".format(relative_angle))
        # print("Clock Angle: {}".format(clock_angle))
        # print("Distance: {}".format(distance_perc))

    items = sorted(items, key=lambda x: x[1])
    
    return items

In [30]:
def GetEnemies(original_img):
    image_np = np.array(original_img)
    
    # Sizing Img Down
    scale_fraction = 0.4
    width = int(image_np.shape[1] * scale_fraction)
    height = int(image_np.shape[0] * scale_fraction)
    dim = (width, height)
    image_np = cv2.resize(image_np, dim)

    image_player_np = copy.deepcopy(image_np)

    for i in range(image_player_np.shape[0]):  # Iterate over rows (height)
        for j in range(image_player_np.shape[1]):  # Iterate over columns (width)
            # If the i, j are 10 pixels away or less from the player center, ignore them.
            if np.sqrt((i - player_circle_center[0])**2 + (j - player_circle_center[1])**2) <= 10:
                image_player_np[i, j] = np.zeros(3, dtype=np.uint8)
            else:
                pixel = image_player_np[i, j]
        
                if pixel[2] > 100 and pixel[2] > pixel[1]*1.4 and pixel[0] < 20:
                    image_player_np[i, j] = np.array([255, 255, 255])
                else: 
                    image_player_np[i, j] = np.zeros(3, dtype=np.uint8)  # Black
    
    player_image = cv2.cvtColor(image_player_np, cv2.COLOR_BGR2GRAY)

    _, player_binary_image = cv2.threshold(player_image, 100, 255, cv2.THRESH_BINARY)

    player_labeled_mask, _ = ndimage.label(player_binary_image)

    player_regions = measure.regionprops(player_labeled_mask)
    
    min_area = 200
    
    closest_region_dist = np.inf
    closest_region_center = (0, 0)
    
    # Loop through region properties and filter out non-circular regions
    for region in player_regions:
        if region.area > min_area and region.perimeter > 0:
            centroid = region.centroid
            region_dist = np.sqrt((centroid[0] - player_circle_center[0])**2 + (centroid[1] - player_circle_center[1])**2)
            if region_dist < closest_region_dist:
                closest_region_dist = region_dist
                closest_region_center = centroid

    # Getting the view area center relative to the player.
    relative_center = (player_circle_center[0] - closest_region_center[0], closest_region_center[1] - player_circle_center[1])

    # Measuring the angle the player is looking
    angle_radians = math.atan2(relative_center[0], relative_center[1])
    player_direction_angle = math.degrees(angle_radians)
    if player_direction_angle < 0:
        player_direction_angle += 360
    
    image_enemy_np = copy.deepcopy(image_np)
    
    for i in range(image_enemy_np.shape[0]):  # Iterate over rows (height)
        for j in range(image_enemy_np.shape[1]):  # Iterate over columns (width)
            pixel = image_enemy_np[i, j]
    
            if pixel[0] > 100 and pixel[0] > pixel[1]*2 and pixel[0] > pixel[2]*2:
                image_enemy_np[i, j] = np.array([255, 255, 255])
            else: 
                image_enemy_np[i, j] = np.zeros(3, dtype=np.uint8)  # Black
    
    image = cv2.cvtColor(image_enemy_np, cv2.COLOR_BGR2GRAY)
    
    # Check if the image is already in binary form, if not, apply a threshold
    # Assuming white pixel values are close to 255
    _, binary_image = cv2.threshold(image, 100, 255, cv2.THRESH_BINARY)
    
    # Label the objects in the binary image
    labeled_mask, _ = ndimage.label(binary_image)
    
    # Find the properties of the labeled regions
    regions = measure.regionprops(labeled_mask)
    
    # Define a range for the circularity metric (0 for a line, 1 for a perfect circle)
    circularity_threshold = 0.3
    
    min_radius = 3
    
    regions_centroids = []
    
    # Loop through region properties and filter out non-circular regions
    for region in regions:
        if region.area > 0 and region.perimeter > 0:
            # To measure the circularity of the region, we'll do the following:
            # Step 1, get all the points in the region.
            centroid = region.centroid
            coords = np.array(region.coords)
    
            # Step 2: Get the furthest point from the center.
            biggest_r = 0
            furthest_point = ()
            for coord in coords:
                distance = abs(centroid[0] - coord[0]) + abs(centroid[1] - coord[1])
                if distance > biggest_r:
                    biggest_r = distance
                    furthest_point = coord
    
            # Step 3: Compare the region area with a perfect circle area.
            circle_area = np.pi * (biggest_r ** 2)
    
            # Step 4: Calculate circularity
            circularity = region.area/circle_area
            
            # If the region is within the circularity thresholds and has at least the minimum diameter, keep it
            if circularity_threshold <= circularity and biggest_r >= min_radius:
                regions_centroids.append(region.centroid)

    enemies = []
    
    for centroid in regions_centroids:
        relative_centroid = (player_circle_center[0] - centroid[0], centroid[1] - player_circle_center[1])

        # Getting the item angle.
        angle_radians = math.atan2(relative_centroid[0], relative_centroid[1])
        direction_angle = math.degrees(angle_radians)
        if direction_angle < 0:
            direction_angle += 360

        # Getting the item's angle relative to the player
        relative_angle = direction_angle - player_direction_angle
        if relative_angle < 0:
            relative_angle += 360

        # Convert the angle to a clock angle (1-12)
        clock_angle = toClockAngles(relative_angle)

        # Getting the distance to enemy
        distance = np.sqrt((relative_centroid[0])**2 + (relative_centroid[1])**2)
        distance_perc = int((distance/furthest_point_distance) * 100)

        # Adding the enemy to the enemies array.
        enemies.append((clock_angle, distance_perc))

        # print("Item Center: {}".format(centroid))
        # print("Item Angle: {}".format(direction_angle))
        # print("Relative Angle: {}".format(relative_angle))
        # print("Clock Angle: {}".format(clock_angle))
        # print("Distance: {}".format(distance_perc))

    enemies = sorted(enemies, key=lambda x: x[1])
    
    return player_direction_angle, enemies

In [31]:
standards = []
standards.append(np.load("first.npy"))
standards.append(np.load("second.npy"))
standards.append(np.load("third.npy"))

def GetChatPromptCode(original_img):
    image_np = np.array(original_img)
    sections = []

    sections.append(image_np[:, :16, :])
    sections.append(image_np[:, 16:32, :])
    sections.append(image_np[:, 32:, :])

    command = ""

    for section, standard in zip(sections, standards):
        similar_pixels = 0
        for i in range(section.shape[0]):  # Iterate over rows (height)
            for j in range(section.shape[1]):  # Iterate over columns (width)
                r, g, b = section[i, j]
                r = int(r)
                g = int(g)
                b = int(b)
    
                if abs(r - g) < 20 and abs(b - r) > 40 and r > 70 and b < 100:
                    if standard[i,j] == 1:
                        similar_pixels += 1
                    else:
                        similar_pixels -= 1
                else:
                    if standard[i,j] == 1:
                        similar_pixels -= 1
        similarity = max(0, similar_pixels/np.sum(standard == 1))
        if similarity > 0.7:
            command += "Z"
        else:
            command += " "
                    
    return command

FileNotFoundError: [Errno 2] No such file or directory: 'first.npy'

#### Testing the chat detection system.
Write in the chat one of the commands. For example "Z Z", and run the following cell to make sure it's reading the chat properly.

In [15]:
img = ImageGrab.grab(bbox=chatcoords)
GetChatPromptCode(img)

'ZZZ'

In [32]:
from gtts import gTTS
import uuid
from playsound import playsound
import pyautogui

def text_to_speech(text):
    # Create a gTTS object
    tts = gTTS(text=text, lang='en')
    filename = "output_{}.mp3".format(str(uuid.uuid1()))

    # Save the speech to an MP3 file
    tts.save(filename)

    pyautogui.mouseDown()
    # Play the audio file (this will play through the default audio device)
    playsound(filename)
    pyautogui.mouseUp()
    
    # Clean up
    os.remove(filename)

In [33]:
import pyperclip
def switchTo(uname):
    comnd = "switch {}".format(uname)
    pyperclip.copy(comnd)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(0.1)
    pyautogui.press('enter')

def flashBooster(booster_name):
    comnd = "flash {}".format(booster_name)
    pyperclip.copy(comnd)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(0.1)
    pyautogui.press('enter')
    time.sleep(0.1)
    pyautogui.press('d')
    time.sleep(0.1)
    pyautogui.press('enter')
    time.sleep(0.1)

def getDropShip():
    comnd = "walkie"
    pyperclip.copy(comnd)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(0.1)
    pyautogui.press('enter')
    time.sleep(0.1)
    pyautogui.press('c')
    time.sleep(0.1)
    pyautogui.press('enter')

def sendMessage(msgs):
    pydirectinput.keyDown('esc')
    time.sleep(0.1)  # Hold the key for a short duration
    pydirectinput.keyUp('esc')
    time.sleep(0.5)

    for msg in msgs:
        if msg == "":
            continue
        pydirectinput.keyDown('/')
        time.sleep(0.1)  # Hold the key for a short duration
        pydirectinput.keyUp('/')
        
        pyperclip.copy(msg)
        pyautogui.hotkey('ctrl', 'v')
        time.sleep(0.1)
    
        pydirectinput.keyDown('enter')
        time.sleep(0.1)  # Hold the key for a short duration
        pydirectinput.keyUp('enter')
        time.sleep(0.2)

    pydirectinput.keyDown('e')
    time.sleep(0.1)  # Hold the key for a short duration
    pydirectinput.keyUp('e')
    time.sleep(1)

    switchTo("SyntaxeAI")
    time.sleep(0.5)

In [34]:
import pydirectinput
import vgamepad as vg

# Optional: Fail-safes to prevent your script from running amok
pydirectinput.FAILSAFE = True

In [35]:
# Note: This function is very inconsistent. You'll need to tinker with the sleep times. The version of this function in AI3 is better.
def charge_walkie():
    pydirectinput.keyDown('esc')
    time.sleep(0.1)  # Hold the key for a short duration
    pydirectinput.keyUp('esc')

    # Create a virtual gamepad (Xbox 360 type by default)
    gamepad = vg.VX360Gamepad()
    time.sleep(2)
    # Move right joystick to the right (X-axis maximum, Y-axis neutral)
    gamepad.right_joystick(x_value=-10000, y_value=0)
    gamepad.update()

    # Wait for a bit
    time.sleep(0.9)

    # Reset joystick to neutral position
    gamepad.right_joystick(x_value=0, y_value=0)
    gamepad.update()
    time.sleep(0.1)

    pydirectinput.keyDown('e')
    time.sleep(0.1)  # Hold the key for a short duration
    pydirectinput.keyUp('e')
    time.sleep(2)

    gamepad.right_joystick(x_value=10000, y_value=0)
    gamepad.update()

    # Wait for a bit
    time.sleep(1)

    # Reset joystick to neutral position
    gamepad.right_joystick(x_value=0, y_value=0)
    gamepad.update()
    time.sleep(0.1)

    # Close the gamepad
    gamepad.reset()
    gamepad.update()

### Instructions:
    1- In game, place the terminal parallel to the charging coil so that the charging coil is to your right when you're on the terminal.
    2- Pick a walkie and turn it on.
    3- Stand right in front of the terminal, so that all it takes to enter it is to press e (but don't enter it yet).
    4- Run the following cell.
    5- Run the command "ZZZ" to make the AI enter the terminal.

In [34]:
time.sleep(10)
booster_name = "albert"
last_cmd = ""
masked_str = ""
enemies_str = ""
items_str = ""
flashing = False
on_terminal = False
while True:
    if on_terminal:
        if flashing:
            flashBooster(booster_name)
        else:
            items = []
            enemies = []
            img = ImageGrab.grab(bbox=sscoords)
            masked_str = ""
            enemies_str = ""
            items_str = ""
            player_direction_angle, enemies = GetEnemies(img)
            items = GetItems(img, player_direction_angle)
            time.sleep(0.1)
            enemies_str = "Enemies {}. ".format(len(enemies))
            for enemy in enemies:
                enemies_str += "{} {}. ".format(enemy[0], enemy[1])
        
            items_str = " Items {}. ".format(len(items))
            for item in items:
                items_str += "{} {}. ".format(item[0], item[1])
    
    time.sleep(0.5)
    chat_img = ImageGrab.grab(bbox=chatcoords)
    new_cmd = GetChatPromptCode(chat_img)
    time.sleep(0.5)

    if on_terminal and not flashing:
         sendMessage([enemies_str, items_str])
        
    if new_cmd != last_cmd and new_cmd != "   ":
        print(new_cmd)
        if new_cmd == "Z  ":
            flashing = False
        elif new_cmd == " Z ":
            flashing = True
        elif new_cmd == "  Z":
            getDropShip()
        elif new_cmd == "ZZ ":
            if on_terminal:
                pydirectinput.keyDown('q')
                time.sleep(0.1)  # Hold the key for a short duration
                pydirectinput.keyUp('q')
                time.sleep(1)
            on_terminal = False
            charge_walkie()
        elif new_cmd == " ZZ":
            if not on_terminal:
                pydirectinput.keyDown('q')
                time.sleep(0.1)  # Hold the key for a short duration
                pydirectinput.keyUp('q')
        elif new_cmd == "ZZZ":
            pydirectinput.keyDown('e')
            time.sleep(0.1)  # Hold the key for a short duration
            pydirectinput.keyUp('e')
            time.sleep(1)
            pyperclip.copy("view monitor")
            pyautogui.hotkey('ctrl', 'v')
            time.sleep(0.1)
            pyautogui.press('enter')
            time.sleep(0.1)
            on_terminal = True
        last_cmd = new_cmd

ZZZ
ZZ 
ZZZ


KeyboardInterrupt: 